# 전기 자동차 충전 데이터를 이용한 충전기 추가 설치 예측 모델 개발
## 기간 : 2020년 8월 ~ 2020년 11월
## 프로젝트 내용
###  1. 전기 자동차 데이터로 미래 전기 자동차 증가량을 예측
###  2. 전기 자동차 사용차의 충전 이력을 확인하여 지역별 충전량을 확인
###  3. 예측된 데이터를 이용하여 미래 전기 자동차 충전량을 예측

In [ ]:
import time
import datetime
import numpy as np
import pandas as pd
import warnings
import openpyxl
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [ ]:
# 충전소
df_station = pd.read_csv('csv\charging_station(201015 ~ 16).csv')
# 사용 중 충전기를 사용시작시간 순으로 정렬
df_station = df_station.sort_values(by=['sid'], ascending=True)

# 충전기
df_charger = pd.read_csv('csv\charger(201015 ~ 16).csv')
# 사용 중 충전기를 사용시작시간 순으로 정렬
df_charger = df_charger.sort_values(by=['cid'], ascending=True)


# 삼천리 에버랜드에 충전양 값이 없어서 임의로 100을 넣음
df_charger['amount'].loc[3595:] = 100

# 분당 충전양
p = (df_charger['amount']/60)
df_charger['amount/m'] = p


# 충전 현황
df1 = pd.read_csv(r'csv\usage_status(201006 ~ 14).csv')
df2 = pd.read_csv(r'csv\usage_status(201015 ~ 16).csv')

df = df1.append(df2)
df = df.sort_values(by=['id'], ascending=True)
df = df.rename({'id':'cid'}, axis='columns')
df

c:\users\12\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,id,start_date,start_time,end_date,end_time
0,111100030,2020-09-24 Thu,01:52:08,2020-09-24 Thu,02:07:13
1,111100030,2020-09-24 Thu,06:52:09,2020-09-24 Thu,07:22:16
2,111100030,2020-09-25 Fri,01:52:11,2020-09-25 Fri,02:07:26
3,111100030,2020-09-25 Fri,11:04:54,2020-09-25 Fri,11:52:33
4,111100060,2020-09-24 Thu,02:22:15,2020-09-24 Thu,02:37:09
...,...,...,...,...,...
9762,ME19C4053,2020-09-25 Fri,10:07:41,\t,\t
9763,ME19C4080,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16
9764,ME19C4081,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16
9765,ME19C4082,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16


In [ ]:
#  끝나는 시간이 아직 나오지 않은 행들을 찾기

a = []
for i in range(0,len(df['end_date'])) :
    if df['end_date'][i] == '\\t' :
        a.append(i)
        
# 위에 거 찾는 간단한 방법

test = df[(df['end_date'] == '\\t')].index
test

Int64Index([  38,   54,   73,   74,   87,  137,  139,  164,  194,  210,
            ...
            9701, 9704, 9707, 9722, 9728, 9734, 9759, 9760, 9761, 9762],
           dtype='int64', length=446)

In [ ]:
# 끝나는 시간이 없는 행들을 없앰
df = df.drop(a)


# 새로 인덱스를 부여
df = df.reset_index(drop=False, inplace=False)


# 원래 있던 인덱스가 인덱스 행이 되어서 없앰
df = df.drop(['index'], axis=1)
df

,id,start_date,start_time,end_date,end_time
0,111100030,2020-09-24 Thu,01:52:08,2020-09-24 Thu,02:07:13
1,111100030,2020-09-24 Thu,06:52:09,2020-09-24 Thu,07:22:16
2,111100030,2020-09-25 Fri,01:52:11,2020-09-25 Fri,02:07:26
3,111100030,2020-09-25 Fri,11:04:54,2020-09-25 Fri,11:52:33
4,111100060,2020-09-24 Thu,02:22:15,2020-09-24 Thu,02:37:09
...,...,...,...,...,...
9316,ME19C4043,2020-09-24 Thu,22:07:16,2020-09-24 Thu,22:22:16
9317,ME19C4080,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16
9318,ME19C4081,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16
9319,ME19C4082,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16


In [ ]:
df['start_day'] = df['start_date'] + ' ' + df['start_time']
df['start_day']   # 충전 시작한 날을 계산하기 쉽게 합쳐버림

df['end_day'] = df['end_date'] + ' ' + df['end_time']
df['end_day']   # 충전 끝나는 날 시간도 마찬가지


# 시작, 끝날을 계산 할 수 있게 타입을 데이트타임으로 만듬
df['start_day'] =  pd.to_datetime(df['start_day'], 
                                  format='%Y-%m-%d %a %H:%M:%S')

# 끝나는 날도 마찬가지로 함
df['end_day'] =  pd.to_datetime(df['end_day'],
                                format='%Y-%m-%d %a %H:%M:%S')


# 충전기가 충전하는데 걸린 시간 계산
overtime =df['end_day'] - df['start_day'] 



# 충전량을 계산하기위해 걸린 시간만 뽑아내기
a = []

for i in range(len(overtime)) :
    test=str(overtime[i])
    test=test.split(' ')
    test=test[2].split(':')
    a.append(test[0]+':'+test[1])

    
# 걸린 시간을 넣음
df['over'] = a
df

,id,start_date,start_time,end_date,end_time,start_day,end_day,over
0,111100030,2020-09-24 Thu,01:52:08,2020-09-24 Thu,02:07:13,2020-09-24 01:52:08,2020-09-24 02:07:13,00:15
1,111100030,2020-09-24 Thu,06:52:09,2020-09-24 Thu,07:22:16,2020-09-24 06:52:09,2020-09-24 07:22:16,00:30
2,111100030,2020-09-25 Fri,01:52:11,2020-09-25 Fri,02:07:26,2020-09-25 01:52:11,2020-09-25 02:07:26,00:15
3,111100030,2020-09-25 Fri,11:04:54,2020-09-25 Fri,11:52:33,2020-09-25 11:04:54,2020-09-25 11:52:33,00:47
4,111100060,2020-09-24 Thu,02:22:15,2020-09-24 Thu,02:37:09,2020-09-24 02:22:15,2020-09-24 02:37:09,00:14
...,...,...,...,...,...,...,...,...
9316,ME19C4043,2020-09-24 Thu,22:07:16,2020-09-24 Thu,22:22:16,2020-09-24 22:07:16,2020-09-24 22:22:16,00:15
9317,ME19C4080,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16,2020-09-23 16:22:26,2020-09-23 19:07:16,02:44
9318,ME19C4081,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16,2020-09-23 16:22:26,2020-09-23 19:07:16,02:44
9319,ME19C4082,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16,2020-09-23 16:22:26,2020-09-23 19:07:16,02:44


In [ ]:
# 걸린 시간을 분단위로 계산하기위해 숫자화 시킴
time = pd.to_numeric(df['over'], errors='ignore')


# 걸린 시간을 분 단위로 다 바꿈
t = []


for i in range(len(time)) :
    test = time[i]
    test = (int(test[1])-(int(test[1])-1))*60 + (int(test[3])*10) + (int(test[4]))
    t.append(test)

    
# 걸린 시간을 분단위로 만든 거 넣음
df['over'] = t
df

,id,start_date,start_time,end_date,end_time,start_day,end_day,over
0,111100030,2020-09-24 Thu,01:52:08,2020-09-24 Thu,02:07:13,2020-09-24 01:52:08,2020-09-24 02:07:13,75
1,111100030,2020-09-24 Thu,06:52:09,2020-09-24 Thu,07:22:16,2020-09-24 06:52:09,2020-09-24 07:22:16,90
2,111100030,2020-09-25 Fri,01:52:11,2020-09-25 Fri,02:07:26,2020-09-25 01:52:11,2020-09-25 02:07:26,75
3,111100030,2020-09-25 Fri,11:04:54,2020-09-25 Fri,11:52:33,2020-09-25 11:04:54,2020-09-25 11:52:33,107
4,111100060,2020-09-24 Thu,02:22:15,2020-09-24 Thu,02:37:09,2020-09-24 02:22:15,2020-09-24 02:37:09,74
...,...,...,...,...,...,...,...,...
9316,ME19C4043,2020-09-24 Thu,22:07:16,2020-09-24 Thu,22:22:16,2020-09-24 22:07:16,2020-09-24 22:22:16,75
9317,ME19C4080,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16,2020-09-23 16:22:26,2020-09-23 19:07:16,104
9318,ME19C4081,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16,2020-09-23 16:22:26,2020-09-23 19:07:16,104
9319,ME19C4082,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16,2020-09-23 16:22:26,2020-09-23 19:07:16,104


In [ ]:
# 콜론 명이 달라서 변경함
df = df.rename({'id':'cid'}, axis='columns')


# cid 값을 기준으로 데이터 프레임을 합침
merge_inner = pd.merge(df, df_charger, on='cid')


# 걸린 시간(분)과 (분당) 충전량을 계산
power = merge_inner['over'] * merge_inner['amount/m']


# 충전량의 한계가 다 64키로와트인지 확인
for i in range(0,len(power)):
    if power[i] > 64:
        power[i] = 64

df['power'] = power

df

,cid,start_date,start_time,end_date,end_time,start_day,end_day,over,power
0,111100030,2020-09-24 Thu,01:52:08,2020-09-24 Thu,02:07:13,2020-09-24 01:52:08,2020-09-24 02:07:13,75,62.500000
1,111100030,2020-09-24 Thu,06:52:09,2020-09-24 Thu,07:22:16,2020-09-24 06:52:09,2020-09-24 07:22:16,90,64.000000
2,111100030,2020-09-25 Fri,01:52:11,2020-09-25 Fri,02:07:26,2020-09-25 01:52:11,2020-09-25 02:07:26,75,62.500000
3,111100030,2020-09-25 Fri,11:04:54,2020-09-25 Fri,11:52:33,2020-09-25 11:04:54,2020-09-25 11:52:33,107,64.000000
4,111100060,2020-09-24 Thu,02:22:15,2020-09-24 Thu,02:37:09,2020-09-24 02:22:15,2020-09-24 02:37:09,74,61.666667
...,...,...,...,...,...,...,...,...,...
9316,ME19C4043,2020-09-24 Thu,22:07:16,2020-09-24 Thu,22:22:16,2020-09-24 22:07:16,2020-09-24 22:22:16,75,64.000000
9317,ME19C4080,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16,2020-09-23 16:22:26,2020-09-23 19:07:16,104,64.000000
9318,ME19C4081,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16,2020-09-23 16:22:26,2020-09-23 19:07:16,104,64.000000
9319,ME19C4082,2020-09-23 Wed,16:22:26,2020-09-23 Wed,19:07:16,2020-09-23 16:22:26,2020-09-23 19:07:16,104,64.000000


In [ ]:
# 지역 단위의 충전량을 알고 싶으니 지역을 데려옴
merge_inners = pd.merge(merge_inner, df_station, on='sid')


# 그렇게 가져온 걸 넣음
df['do']=merge_inners['do']
df['city']=merge_inners['city']
df['gu']=merge_inners['gu']

df = df.drop(['start_time', 'end_time'], axis=1)
df

,cid,start_date,end_date,start_day,end_day,over,power,do,city,gu
0,111100030,2020-09-24 Thu,2020-09-24 Thu,2020-09-24 01:52:08,2020-09-24 02:07:13,75,62.500000,서울특별시,서울시,종로구
1,111100030,2020-09-24 Thu,2020-09-24 Thu,2020-09-24 06:52:09,2020-09-24 07:22:16,90,64.000000,서울특별시,서울시,종로구
2,111100030,2020-09-25 Fri,2020-09-25 Fri,2020-09-25 01:52:11,2020-09-25 02:07:26,75,62.500000,서울특별시,서울시,종로구
3,111100030,2020-09-25 Fri,2020-09-25 Fri,2020-09-25 11:04:54,2020-09-25 11:52:33,107,64.000000,서울특별시,서울시,종로구
4,111100060,2020-09-24 Thu,2020-09-24 Thu,2020-09-24 02:22:15,2020-09-24 02:37:09,74,61.666667,서울특별시,서울시,종로구
...,...,...,...,...,...,...,...,...,...,...
9316,ME19C4043,2020-09-24 Thu,2020-09-24 Thu,2020-09-24 22:07:16,2020-09-24 22:22:16,75,64.000000,대구광역시,대구시,수성구
9317,ME19C4080,2020-09-23 Wed,2020-09-23 Wed,2020-09-23 16:22:26,2020-09-23 19:07:16,104,64.000000,부산광역시,부산시,강서구
9318,ME19C4081,2020-09-23 Wed,2020-09-23 Wed,2020-09-23 16:22:26,2020-09-23 19:07:16,104,64.000000,부산광역시,부산시,강서구
9319,ME19C4082,2020-09-23 Wed,2020-09-23 Wed,2020-09-23 16:22:26,2020-09-23 19:07:16,104,64.000000,부산광역시,부산시,강서구


In [ ]:
# 지역 단위로 그룹해서
group = df.groupby(['do'])
print(group)

# 그룹한 것 더함
dff = group.sum()
dff = dff.reset_index(drop=False, inplace=False)
# 지역별 충전기 수
dff = dff.drop(['over'], axis=1)
dff['충전량'] = round(dff['power'],2)
dff

,do,power,충전량
0,강원도,26362.666667,26362.67
1,경기도,82045.833333,82045.83
2,경상남도,39353.833333,39353.83
3,경상북도,65026.333333,65026.33
4,광주광역시,12047.833333,12047.83
5,대구광역시,18633.166667,18633.17
6,대전광역시,21883.000000,21883.00
7,부산광역시,16274.166667,16274.17
8,서울특별시,47165.500000,47165.50
9,세종특별자치시,3187.833333,3187.83


In [ ]:
# 전기 차 2020년 10월 데이터와 2021년 10월 데이터이다.

ff = pd.DataFrame({'do':['서울특별시', '부산광역시', '대구광역시', '인천광역시',
                         '광주광역시','대전광역시', '울산광역시', '세종특별자치시',
                         '경기도', '강원도', '충청북도', '충청남도', '전라북도',
                         '전라남도', '경상북도', '경상남도', '제주특별자치도'],
                   '현 차량':[21925.844, 4671.588, 13714.723, 4254.6475, 3137.5955, 3506.7373,
                           2265.3184, 1132.5188, 17789.883, 3219.52, 3470.489, 4328.225,
                           2987.9155, 4654.3276, 5573.46, 5187.2515, 20673.354],
                   '미래 차량':[33836.79, 7309.6553, 19102.797, 6794.291, 4320.6143, 
                            5149.456, 3647.2522, 1508.9034, 27238.555, 4515.1396, 5147.6577,
                            6606.9688, 4619.658, 6579.838, 8131.8984, 7484.1636, 24638.646]})


# 17개 도별 증가량 구함
ff['증가량'] = ff['미래 차량'] / ff['현 차량']


# 충전기에 지역 정보를 넣기 위해 합침
ee = pd.merge(df_charger, df_station, on='sid')
ee = pd.merge(ee, dff, on='do')

ee = ee.drop(['amount', 'amount/m', 'power'], axis=1)

# 지역별 충전기 대수를 알기 위해 임의로 1를 넣음
ee['충전기 수'] = 1


# 지역 단위로 그룹해서
grouped = ee.groupby('do')
print(grouped)

# 그룹한 것 더함
d = grouped.sum()
d = d.reset_index(drop=False, inplace=False)
# 지역별 충전기 수
d

,do,충전량,충전기 수
0,강원도,8488779.74,322
1,경기도,35361752.73,431
2,경상남도,10310703.46,262
3,경상북도,27701216.58,426
4,광주광역시,807204.61,67
5,대구광역시,1937849.68,104
6,대전광역시,2078885.00,95
7,부산광역시,1285659.43,79
8,서울특별시,11697044.00,248
9,세종특별자치시,102010.56,32


In [ ]:
# 구했으니 이걸 또 합침
h = pd.merge(d, ff, on='do')

# 이제 1 항목에 있는 지역별 충전기 수가
# 차량 증가량에 따라 얼마나 증가해야하는지 계산함
h['미래 충전량'] = h['충전량'] / h['현 차량'] * h['미래 차량']
h['미래 충전기 수'] = h['미래 충전량'] / (h['충전량'] / h['충전기 수'])
h
#data = h.sort_values(["충전기증가"], ascending=[False])
#data

,do,충전량,충전기 수,현 차량,미래 차량,증가량,미래 충전량,미래 충전기 수
0,강원도,8488779.74,322,3219.5200,4515.1396,1.402426,1.190489e+07,451.581276
1,경기도,35361752.73,431,17789.8830,27238.5550,1.531126,5.414330e+07,659.915369
2,경상남도,10310703.46,262,5187.2515,7484.1636,1.442799,1.487628e+07,378.013455
3,경상북도,27701216.58,426,5573.4600,8131.8984,1.459040,4.041717e+07,621.550835
4,광주광역시,807204.61,67,3137.5955,4320.6143,1.377046,1.111558e+06,92.262103
5,대구광역시,1937849.68,104,13714.7230,19102.7970,1.392868,2.699169e+06,144.858258
6,대전광역시,2078885.00,95,3506.7373,5149.4560,1.468446,3.052731e+06,139.502414
7,부산광역시,1285659.43,79,4671.5880,7309.6553,1.564705,2.011677e+06,123.611665
8,서울특별시,11697044.00,248,21925.8440,33836.7900,1.543238,1.805132e+07,382.722960
9,세종특별자치시,102010.56,32,1132.5188,1508.9034,1.332343,1.359130e+05,42.634973


In [ ]:
# 증가량은 미래 차량에서 현 차량을 나누어 증가한 비율을 나타낸 것이다.
# 미래 충전기 수는 차량 증가량과 충전기의 충전량 비율에 따라 미래 충전기 수를 예측한 것이다.

In [ ]:
data.to_csv('sample.csv')

NameError: name 'data' is not defined